In [1]:
# Model
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from langchain_huggingface import HuggingFacePipeline
import torch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import PromptTemplate
# Vector stores
import fitz  # PyMuPDF
import pdfplumber
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader, PDFPlumberLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, KonlpyTextSplitter, MarkdownHeaderTextSplitter, MarkdownTextSplitter
from langchain_community.retrievers import BM25Retriever, KNNRetriever
from langchain.retrievers import EnsembleRetriever
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader, PDFPlumberLoader ,UnstructuredPDFLoader
from langchain_community.retrievers import BM25Retriever, KNNRetriever 
from langchain.retrievers import EnsembleRetriever
from langchain_teddynote.retrievers import KiwiBM25Retriever, OktBM25Retriever
from langchain.docstore.document import Document
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import unicodedata
import pymupdf4llm
import time
import re
from konlpy.tag import Okt
from pdf2image import convert_from_path
import pytesseract
from konlpy.tag import Kkma
# etc
import os
import pandas as pd
from tqdm import tqdm
import unicodedata
import logging
from PyPDF2 import PdfReader
import json
device = 'cuda' if torch.cuda.is_available() else 'cpu'  # GPU 사용 가능 여부 및 MPS 지원 여부 확인
print(device)

c:\ProgramData\anaconda3\envs\llm_project\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


In [2]:
# intfloat/multilingual-e5-small
# jhgan/ko-sroberta-multitask

def get_embedding():
    
    embeddings = HuggingFaceEmbeddings(
        model_name='jhgan/ko-sroberta-multitask',
        model_kwargs={'device': device},
        encode_kwargs={'normalize_embeddings': True})
    return embeddings
def normalize_string(s):
    try:
        normalized = unicodedata.normalize('NFC', s)
        return normalized.encode('utf-8', errors='replace').decode('utf-8')
    except Exception:
        return s
def clean_text(text):
    text = text.replace("�", " ").replace("", " ")  # 잘못된 인코딩 문자 제거
    return text

def format_docs(docs):
    context = ""
    i = 1
    for doc in docs:
        #context += f"Document: {i}" +"Source:"+ doc.metadata['source'] +'\n'
        context += doc.page_content
        context += '\n---\n'
        i += 1
    return context


In [3]:

def get_docs(pdf_path):
    documents = []
    try:
        # 페이지 단위로 pymupdf4llm 사용
        md_read = pymupdf4llm.to_markdown(pdf_path,page_chunks=True)
        total_pages = len(md_read)    
        
        for page_data in md_read:
            page_number = page_data.get('metadata', {}).get('page', None)
            text = clean_text(normalize_string(page_data['text']))
            metadata = {
                "Source_path": pdf_path,
                "page_number": page_number,
                "total_pages": total_pages,
                "tables": page_data.get('tables', None),
            }
            
            documents.append(Document(page_content=text, metadata=metadata))
    except Exception as e:
        logging.error(f"Failed to process PDF with pymupdf4llm: {e}")

    return documents

def chunk_documents(docs):
    markdown_headers = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
        ("####", "Header 4")
    ]
    
    # 마크다운 스플리터 설정
    markdown_splitter = MarkdownTextSplitter(
        chunk_size=2000,
        chunk_overlap=400,
    )
    
    chunks = []
    for doc in docs:
        text = doc.page_content
        metadata = doc.metadata
        md_header_splits = markdown_splitter.split_text(text)
        for md_split in md_header_splits:   
        
            # 테이블이 있는 페이지는 청크로 분할하지 않음
            chunks.append(Document(page_content=md_split, metadata = metadata))

            
               

    return chunks

def make_db(df):
    documents = []
    chunks = []
    pdf_files = df['Source_path'].unique()
    for pdf_file in tqdm(pdf_files):
        # 문서 로드
        documents.extend(get_docs(pdf_file))
        # 청크 생성
        #chunks.extend(chunk_documents(documents))
    print(f"Total number of documents: {len(documents)}")
    
    # print(f"Total number of chunks: {len(chunks)}")

    faiss = FAISS.from_documents(documents, embedding=get_embedding())
    knn = KNNRetriever.from_documents(documents, embeddings=get_embedding())
    bm = OktBM25Retriever.from_documents(documents)

    return faiss,knn, bm

In [4]:
def fewshot_db(df):
    df = df.drop('SAMPLE_ID', axis=1)
    df = df.drop('Source_path', axis=1)
    df = df.to_dict(orient='records')
    print("Loaded Fewshot Set:", len(df))
    to_vectorize = ["\n\n".join(normalize_string(value) for value in example.values()) for example in df]
    faiss = FAISS.from_texts(to_vectorize, embedding=get_embedding())
    # bm = BM25Retriever.from_texts(to_vectorize)
    # knn = KNNRetriever.from_texts(to_vectorize, embeddings=get_embedding())
    return faiss

In [5]:
train_df = pd.read_csv('train.csv', encoding='utf-8')
test_df = pd.read_csv('test.csv', encoding='utf-8')

In [6]:
def format_docs(docs):
    """검색된 문서들을 하나의 문자열로 포맷팅"""
    context = ""
    for i, doc in enumerate(docs):
        #context += f"Document {i+1}\n"
        doc.page_content = doc.page_content.replace("{", "(")
        doc.page_content = doc.page_content.replace("}", ")")
        
        context += doc.page_content
        context += '\n\n'
    return context.strip()

In [7]:
def setup_llm_pipeline(model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"):
    # 토크나이저 로드 및 설정
        # 양자화 설정 적용
    bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, 
    bnb_4bit_use_double_quant=True, 
    bnb_4bit_quant_type="nf4", 
    bnb_4bit_compute_dtype=torch.bfloat16
    )
    model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config,low_cpu_mem_usage=True)
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    text_generation_pipeline = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        return_full_text=False,
        max_new_tokens=1024,
        eos_token_id = terminators,
        pad_token_id = tokenizer.eos_token_id
    )

    llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

    return llm
# ghost-x/ghost-8b-beta-1608
# OpenBuddy/openbuddy-llama3.1-8b-v22.3-131k
llm = setup_llm_pipeline()

Loading checkpoint shards: 100%|██████████| 4/4 [00:18<00:00,  4.70s/it]


In [8]:
from collections import Counter
def calculate_f1_score(true_sentence, predicted_sentence, sum_mode=True):

    #공백 제거
    true_sentence = ''.join(true_sentence.split())
    predicted_sentence = ''.join(predicted_sentence.split())
    
    true_counter = Counter(true_sentence)
    predicted_counter = Counter(predicted_sentence)

    #문자가 등장한 개수도 고려
    if sum_mode:
        true_positive = sum((true_counter & predicted_counter).values())
        predicted_positive = sum(predicted_counter.values())
        actual_positive = sum(true_counter.values())

    #문자 자체가 있는 것에 focus를 맞춤
    else:
        true_positive = len((true_counter & predicted_counter).values())
        predicted_positive = len(predicted_counter.values())
        actual_positive = len(true_counter.values())

    #f1 score 계산
    precision = true_positive / predicted_positive if predicted_positive > 0 else 0
    recall = true_positive / actual_positive if actual_positive > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return precision, recall, f1_score

def calculate_average_f1_score(true_sentences, predicted_sentences):
    
    total_precision = 0
    total_recall = 0
    total_f1_score = 0
    
    for true_sentence, predicted_sentence in zip(true_sentences, predicted_sentences):
        precision, recall, f1_score = calculate_f1_score(true_sentence, predicted_sentence)
        total_precision += precision
        total_recall += recall
        total_f1_score += f1_score
    
    avg_precision = total_precision / len(true_sentences)
    avg_recall = total_recall / len(true_sentences)
    avg_f1_score = total_f1_score / len(true_sentences)
    
    return {
        'average_precision': avg_precision,
        'average_recall': avg_recall,
        'average_f1_score': avg_f1_score
    }

In [9]:

def extract_answer(response):
    # AI: 로 시작하는 줄을 찾아 그 이후의 텍스트만 추출
    lines = response.split('\n')
    for line in lines:
        line = line.replace('**', '')
        if line.startswith('Answer:'):
            return line.replace('Answer:', '').strip()
        if line.startswith('assistant:'):
            return line.replace('assistant:', '').strip()
    return response.strip()  # AI: 를 찾지 못한 경우 전체 응답을 정리해서 반환

def equal_path(contexts, source_path):
    adjusted_docs = []
    for doc in contexts:
        if doc.metadata['Source_path'] == source_path:
            adjusted_docs.append(doc)
    return adjusted_docs

def rerun(question,context,answer,llm,num_repeat):
    full_template = "<|begin_of_text|>"
    full_template += """<|start_header_id|>system<|end_header_id|>당신은 이전 답변을 검증하는 챗봇입니다. 질문과 문맥, 이전 답변을 참고해서 지시사항을 따르세요. 지시사항을 따를 때 서론 없이 출력하세요.<|eot_id|>"""
    full_template += f"""<|start_header_id|>user<|end_header_id|>Question: {question} \n\nContexts: {context} \n\nPrevious Answer: {answer} \n\n"""
    full_template += """{input}<|eot_id|>"""
    full_template += """<|start_header_id|>assistant<|end_header_id|>"""
    
    prompt = PromptTemplate(template=full_template)
    chain = (
    {
        "input": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
    )
    return chain.invoke("핵심 단어들을 바탕으로, 한 문장으로 요약하세요. 만약 한 문장이라면 그대로 출력하세요.")
    

def run (retriever,dataset,llm,k=3,verbose=False):
    results = []
    for i, row in (dataset.iterrows()):
        full_template = "<|begin_of_text|>"
        full_template += """<|start_header_id|>system<|end_header_id|>
당신은 유용한 금융 정보 QnA 챗봇입니다.
질문을 차근차근 생각하고, 답변 시 반드시 문맥 정보를 활용해야합니다. 
객관적이고 공식적인 문체를 사용하세요.
서론 없이 핵심 내용을 한 문장으로 작성해주세요. 
<|eot_id|>
"""
        question = row['Question']          
        # full_template += """ """
        contexts = retriever.invoke(normalize_string(question))
        contexts = equal_path(contexts,row['Source_path'])
        contexts = format_docs(contexts)
        full_template += """<|start_header_id|>user<|end_header_id|>Question: {input}\n\n"""
        full_template += f"""Contexts: {contexts}<|eot_id|>"""
        full_template += """<|start_header_id|>assistant<|end_header_id>"""
        
        prompt = PromptTemplate(template=full_template, input_variables=["input"])
        qa_chain = (
        {
            "input": RunnablePassthrough(),
        }
        | prompt
        | llm
        | StrOutputParser()
        )

        answer = qa_chain.invoke(input=question)
        answer = extract_answer(answer)
        lines = answer.split('\n')
        if  len(lines) > 1:
            previous = answer
            try:
                before = calculate_f1_score(row['Answer'],answer)[2]
            except:
                before = None
            answer = rerun(question=question,
                           context=contexts,
                           answer=answer,
                           llm=llm,
                           num_repeat=1)
        answer = extract_answer(answer)
        results.append({
            "Question": question,
            "Answer": answer,
            "Source": row['Source']
        })
        if verbose:
            print(f"{i}/{len(dataset)}")
            print("Question: ", question, end=" | ")
            print("Context Number |",len(contexts))
            try:
                print(calculate_f1_score(row['Answer'],answer)[2],end=" | ")
            except:
                pass
            print("Answer: ", results[-1]['Answer'])
            try:
                print("Before: ",before," | ",previous)  
                
                previous = None
                before = None
            except:
                pass
            
            try:
                print("REAL Answer: ",row['Answer'])
            except:
                pass
            
            print()

    return results

In [10]:
from sklearn.model_selection import KFold
import copy

weight = [0.3,0.3,0.4]
train_faiss_db,knn_retriever ,train_bm_retrievier = make_db(train_df) 

train_k = 3
train_bm_retrievier.k = train_k
knn_retriever.k = train_k
train_faiss_retriever = train_faiss_db.as_retriever(search_type="mmr",search_kwargs={'k':train_k} )
train_ensemble_retriever = EnsembleRetriever(
    retrievers=[train_bm_retrievier, knn_retriever,train_faiss_retriever], weights=weight , search_kwargs={'k':train_k}
)


fewshot_k = 3

k_folds = 4
fold_results = []
kf = KFold(n_splits=k_folds, shuffle=True, random_state=52)
for fold, (train_index, val_index) in enumerate(kf.split(train_df)):
    fold_result = []
    train_set = train_df.iloc[train_index]
    val_set = train_df.iloc[val_index]

    pred = run(train_ensemble_retriever, val_set, llm, verbose=True)
    result = pd.DataFrame()
    result['pred'] = [result['Answer'] for result in pred]
    val_set.index = range(len(val_set))
    result['gt'] = val_set['Answer']
        
    result = calculate_average_f1_score(result['gt'], result['pred'])
    print(result)
    fold_results.append(result)
    break

100%|██████████| 16/16 [02:23<00:00,  8.99s/it]


Total number of documents: 1078


c:\ProgramData\anaconda3\envs\llm_project\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
c:\ProgramData\anaconda3\envs\llm_project\Lib\site-packages\transformers\models\llama\modeling_llama.py:653: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


1/124
Question:  2024년 중앙정부의 예산 지출은 어떻게 구성되어 있나요? | Context Number | 2938
0.8807339449541284 | Answer:  2024년 중앙정부의 예산 지출은 일반회계 356.5조원, 특별회계 81.7조원으로 총 656.6조원으로 구성되어 있습니다.
REAL Answer:  2024년 중앙정부의 예산 지출은 일반회계 356.5조원, 21개 특별회계 81.7조원으로 구성되어 있습니다.

4/124
Question:  2024년 총수입은 얼마이며, 예산수입과 기금수입은 각각 몇 조원인가요? | Context Number | 4749
1.0 | Answer:  2024년 총수입은 612.2조원이며, 예산수입은 395.5조원, 기금수입은 216.7조원입니다.
REAL Answer:  2024년 총수입은 612.2조원이며, 예산수입은 395.5조원, 기금수입은 216.7조원입니다.

6/124
Question:  2024년의 기금수입은 어떻게 구성되어 있나요? | Context Number | 2559
0.7679999999999999 | Answer:  2024년 기금수입은 216.7조원으로, 사회보장기여금 92.3조원, 융자원금회수 33.8조원, 경상이전수입 39.6조원 등으로 구성됩니다.
REAL Answer:  2024년도 기금수입은 사회보장성기금 92.3조원, 경상이전수입 39.6조원, 기타 84.7조원으로 구성되어 있습니다.

11/124
Question:  2024년 총지출 기준 예산의 일반회계와 특별회계의 비중이 각각 얼마인가? | Context Number | 3054
0.8888888888888888 | Answer:  2024년 총지출 기준 예산의 일반회계와 특별회계의 비중은 각각 54.3%와 12.4%입니다.
REAL Answer:  2024년 총지출 중 일반회계와 특별회계의 비중은 각각 54.3%와 12.4%이다.

19/124
Question:  2020년 결산 기준, 연구개발비 중 

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


37/124
Question:  2007년 시행된 국가재정법에서 추경 편성 사유로 명시된 사항은 무엇인가? | Context Number | 2641
0.9486166007905139 | Answer:  추가경정 편성 사유로 명시된 사항은 ①전쟁이나 대규모 재해가 발생한 경우, ②경기 침체, 대량 실업, 남북 관계의 변화, 경제 협력과 같은 대내외 여건에 중대한 변화가 발생하였거나 발생할 우려가 있는 경우, ③법령에 따라 국가가 지급하여야 하는 지출이 발생하거나 증가하는 경우에 한정됩니다.
REAL Answer:  국가재정법에서 추경 편성 사유로 명시된 사항은 전쟁이나 대규모 재해가 발생한 경우, 경기침체, 대량 실업, 남북 관계의 변화, 경제협력과 같은 대내외 여건에 중대한 변화가 발생하였거나 발생할 우려가 있는 경우, 법령에 따라 국가가 지급하여야 하는 지출이 발생하거나 증가하는 경우에 한정되어 있습니다.

41/124
Question:  2024년 교육재정 교부금의 규모는 얼마이고, 전년 대비 추이는 어떠한가? | Context Number | 4936
0.9166666666666667 | Answer:  2024년 교육재정 교부금은 73.0조원이며, 전년 대비 8.9% 감소했습니다.
REAL Answer:  2024년 교육재정교부금은 73.0조원이며 전년 대비 8.9% 감소하였다.

43/124
Question:  2024년에 교육재정교부금에서 유아교육비 및 보육료 지원에 할당된 비중은? | Context Number | 5253
0.1777777777777778 | Answer:  2024년 교육재정교부금에서 유아교육비 및 보육료 지원에 할당된 비중은 3.5%입니다.
REAL Answer:  4.40%

44/124
Question:  교육재정 교부금이란? | Context Number | 2357
0.8333333333333334 | Answer:  교육재정 교부금이란 국가가 교육기관 및 교육행정기관의 경비를 지원하는 일반회계 보통･특별교부금, 고교

In [11]:
# from save_module import save


# weight = [0.5,0.5]
# test_faiss_db, test_bm_retrievier = make_db(test_df)

# test_k = 3
# test_bm_retrievier.k = test_k
# #test_knn_retriever.k = test_k
# test_faiss_retriever = test_faiss_db.as_retriever(search_type="mmr",search_kwargs={'k':test_k} )
# test_ensemble_retriever = EnsembleRetriever(
#     retrievers=[test_bm_retrievier, test_faiss_retriever], weights=weight
# )


# results = run(test_ensemble_retriever, test_df, llm, verbose=True)
# save(results)